In [1]:
import csv
from tqdm import tqdm
import os

label_dict = {}
with open("../ComParE2017_Cold_4students/lab/ComParE2017_Cold.tsv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f, delimiter="\t")
    rows = list(reader)
    for row in tqdm(rows, desc="Loading labels"):
        label_dict[row["file_name"]] = row["Cold (upper respiratory tract infection)"]

def search_in_ground_truth(file_id: str, label_dict: dict) -> str:
    wav_name = file_id + ".wav"
    return label_dict.get(wav_name, None)

def search_in_labels(filename, label_dict):
    base_name = os.path.splitext(filename)[0]
    
    if "_logmel" in base_name:
        base_name = base_name.replace("_logmel", "")
    if "_flipped" in base_name:
        base_name = base_name.replace("_flipped", "")
    
    parts = base_name.split("_")
    if len(parts) >= 2:
        audio_filename = f"{parts[0]}_{parts[1]}.wav"
    else:
        audio_filename = f"{base_name}.wav"
    
    return label_dict.get(audio_filename, None)

Loading labels: 100%|██████████| 19101/19101 [00:00<00:00, 2926162.41it/s]


In [2]:
import pandas as pd
from tqdm import tqdm

def load_physical_features_as_df():
    df = pd.read_csv("audio_features.csv", delimiter=",", encoding="utf-8")
    
    df_filtered = df[df['filename'].isin(label_dict.keys())]
    
    print(f"📊 Physical features loaded:")
    print(f"  Total rows in CSV: {len(df)}")
    print(f"  Filtered rows: {len(df_filtered)}")
    print(f"  Features: {list(df_filtered.columns)}")
    
    return df_filtered

physical_features_df = load_physical_features_as_df()

📊 Physical features loaded:
  Total rows in CSV: 28652
  Filtered rows: 19101
  Features: ['filename', 'split', 'duration', 'mfcc_0_mean', 'mfcc_0_std', 'mfcc_1_mean', 'mfcc_1_std', 'mfcc_2_mean', 'mfcc_2_std', 'mfcc_3_mean', 'mfcc_3_std', 'mfcc_4_mean', 'mfcc_4_std', 'mfcc_5_mean', 'mfcc_5_std', 'mfcc_6_mean', 'mfcc_6_std', 'mfcc_7_mean', 'mfcc_7_std', 'mfcc_8_mean', 'mfcc_8_std', 'mfcc_9_mean', 'mfcc_9_std', 'mfcc_10_mean', 'mfcc_10_std', 'mfcc_11_mean', 'mfcc_11_std', 'mfcc_12_mean', 'mfcc_12_std', 'chroma_0_mean', 'chroma_0_std', 'chroma_1_mean', 'chroma_1_std', 'chroma_2_mean', 'chroma_2_std', 'chroma_3_mean', 'chroma_3_std', 'chroma_4_mean', 'chroma_4_std', 'chroma_5_mean', 'chroma_5_std', 'chroma_6_mean', 'chroma_6_std', 'chroma_7_mean', 'chroma_7_std', 'chroma_8_mean', 'chroma_8_std', 'chroma_9_mean', 'chroma_9_std', 'chroma_10_mean', 'chroma_10_std', 'chroma_11_mean', 'chroma_11_std', 'spectral_contrast_0_mean', 'spectral_contrast_0_std', 'spectral_contrast_1_mean', 'spectral_

In [3]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os
import torch
import numpy as np

class SpectrogramDataset(Dataset):
    def __init__(self, image_paths, label_dict, physical_features_df, transform=None, is_training=False):
        self.image_paths = image_paths
        self.label_dict = label_dict
        self.is_training = is_training 
        self.physical_features_df = physical_features_df
        
        self.prepare_physical_features()
        
        self.base_transform = transforms.Compose([
            transforms.Resize((210, 70)),
            transforms.RandomAffine(degrees=0, translate=(0.3, 0)),
            transforms.ToTensor()
        ])
        
        self.c_train_transform = transforms.Compose([
            transforms.RandomAffine(degrees=0, translate=(0.3, 0)),
            transforms.Resize((210, 70)),
            transforms.ToTensor()
        ])
        
    def prepare_physical_features(self):
        numeric_columns = self.physical_features_df.select_dtypes(include=[np.number]).columns.tolist()
        
        columns_to_remove = ['split'] if 'split' in numeric_columns else []
        for col in columns_to_remove:
            numeric_columns.remove(col)
        
        self.feature_columns = numeric_columns
        
        self.features_dict = {}
        for _, row in self.physical_features_df.iterrows():
            filename = row['filename']
            features = row[self.feature_columns].values.astype(np.float32)
            features = np.nan_to_num(features, nan=0.0)
            self.features_dict[filename] = torch.tensor(features, dtype=torch.float32)
        
        print(f"📊 Physical features prepared:")
        print(f"  Feature dimensions: {len(self.feature_columns)}")
        
    def get_physical_features(self, image_filename):
        base_name = os.path.splitext(image_filename)[0]
        
        if "_logmel" in base_name:
            base_name = base_name.replace("_logmel", "")
        if "_flipped" in base_name:
            base_name = base_name.replace("_flipped", "")
        
        parts = base_name.split("_")
        if len(parts) >= 2:
            audio_filename = f"{parts[0]}_{parts[1]}.wav"
        else:
            audio_filename = f"{base_name}.wav"
        
        if audio_filename in self.features_dict:
            return self.features_dict[audio_filename]
        else:
            print(f"⚠️ No features found for {audio_filename}, using zero vector")
            return torch.zeros(len(self.feature_columns), dtype=torch.float32)
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        filename = os.path.basename(image_path)
        
        label = search_in_labels(filename, self.label_dict)
        
        image = Image.open(image_path).convert("RGB")

        if self.is_training and label == "C":
            image = self.c_train_transform(image)
        else:
            image = self.base_transform(image)

        label_num = 1 if label == "C" else 0
        
        physical_features = self.get_physical_features(filename)

        return image, physical_features, label_num

In [4]:
import random
def create_progressive_undersampling(image_paths, label_dict, stages=[0.4, 0.6, 0.8]):

    all_datasets = []
    
    for stage_ratio in stages:
        cold_paths = []
        healthy_paths = []
        
        for path in image_paths:
            filename = os.path.basename(path)
            label = search_in_labels(filename, label_dict)
            
            if label == "C":
                cold_paths.append(path)
            elif label == "NC":
                healthy_paths.append(path)
        
        target_healthy = int(len(healthy_paths) * stage_ratio)
        sampled_healthy = random.sample(healthy_paths, target_healthy)
        
        stage_paths = cold_paths + sampled_healthy
        random.shuffle(stage_paths)
        
        stage_dataset = SpectrogramDataset(stage_paths, label_dict, physical_features_df, is_training=True)
        all_datasets.append(stage_dataset)
        
        print(f"📊 Stage {stage_ratio}: {len(cold_paths)} Cold + {len(sampled_healthy)} Healthy")
    
    return all_datasets

In [12]:
import os
import glob
from torch.utils.data import DataLoader

data_split = ["train_files", "devel_files"]
img_dir = "../spectrogram_images/log_mel"  

def collect_image_paths_devel(split_name):
        sub_dir = os.path.join(img_dir, split_name)
        print(f"🔍 Looking for images in: {sub_dir}")
        
        if not os.path.exists(sub_dir):
            print(f"❌ Directory does not exist: {sub_dir}")
            return []
        
        png_files = glob.glob(os.path.join(sub_dir, "*.png"))
        
        filtered_files = [f for f in png_files if "flipped" not in os.path.basename(f)]
        
        print(f"📁 Found {len(png_files)} PNG files in {split_name}")
        print(f"📋 After filtering out 'flipped' files: {len(filtered_files)} files")
        
        return filtered_files

def collect_image_paths(split_name):
    sub_dir = os.path.join(img_dir, split_name)
    print(f"🔍 Looking for images in: {sub_dir}")
    
    if not os.path.exists(sub_dir):
        print(f"❌ Directory does not exist: {sub_dir}")
        return []
    
    png_files = glob.glob(os.path.join(sub_dir, "*.png"))
    print(f"📁 Found {len(png_files)} PNG files in {split_name}")
    
    return png_files

print("🚀 Collecting image paths...")
train_image_paths = collect_image_paths("train_files")
devel_image_paths = collect_image_paths_devel("devel_files")

progressive_datasets = create_progressive_undersampling(
    train_image_paths, 
    label_dict, 
    stages=[0.6, 0.8] 
)

selected_stage = 1 
train_dataset = progressive_datasets[selected_stage]
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
print(f"✅ Created train loader with {len(train_dataset)} samples")

devel_dataset = SpectrogramDataset(devel_image_paths, label_dict, physical_features_df, is_training=False)
test_dataset = SpectrogramDataset(devel_image_paths, label_dict, physical_features_df, is_training=False) 
    
devel_loader = DataLoader(devel_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(devel_dataset, batch_size=512, shuffle=False)
print(f"✅ Created devel loader with {len(devel_dataset)} samples")


🚀 Collecting image paths...
🔍 Looking for images in: ../spectrogram_images/log_mel\train_files
📁 Found 10475 PNG files in train_files
🔍 Looking for images in: ../spectrogram_images/log_mel\devel_files
📁 Found 10607 PNG files in devel_files
📋 After filtering out 'flipped' files: 9596 files
📊 Physical features prepared:
  Feature dimensions: 75
📊 Stage 0.6: 1940 Cold + 5121 Healthy
📊 Physical features prepared:
  Feature dimensions: 75
📊 Stage 0.8: 1940 Cold + 6828 Healthy
✅ Created train loader with 8768 samples
📊 Physical features prepared:
  Feature dimensions: 75
📊 Physical features prepared:
  Feature dimensions: 75
✅ Created devel loader with 9596 samples


In [ ]:
from torch import nn
import torch.nn.functional as F 

class EnhancedCNNBinaryClassifier(nn.Module):
    def __init__(self, input_shape=(3, 300, 100), physical_feature_dim=75, num_classes=1):
        super(EnhancedCNNBinaryClassifier, self).__init__()
        
        self.conv1 = nn.Conv2d(input_shape[0], 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        self.image_feature_dim = 512 * 4 * 4
        self.image_reducer = nn.Sequential(
            nn.Linear(self.image_feature_dim, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        self.physical_processor = nn.Sequential(
            nn.Linear(physical_feature_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2)
        )

        self.attention = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 2),
            nn.Softmax(dim=1)
        )

        self.projection_head = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.LayerNorm(64)
        )

        self.classifier = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, image_input, physical_input):
        x = F.relu(self.bn1(self.conv1(image_input)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        image_feat = self.image_reducer(x)

        phys_feat = self.physical_processor(physical_input)
        
        combined = torch.cat([image_feat, phys_feat], dim=1)  # [batch, 512]
        attention_weights = self.attention(combined)  # [batch, 2]
        
        fused = (attention_weights[:, 0:1] * image_feat + 
                attention_weights[:, 1:2] * phys_feat)

        embedding = self.projection_head(fused)
        embedding = F.normalize(embedding, dim=1)
        
        logits = self.classifier(fused)
        
        return embedding, logits

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SupervisedContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.05, minority_weight=2.0):
        super(SupervisedContrastiveLoss, self).__init__()
        self.temperature = temperature
        self.minority_weight = minority_weight

    def forward(self, features, labels):
        device = features.device
        batch_size = features.shape[0]
        
        labels = labels.contiguous().view(-1, 1)
        mask = torch.eq(labels, labels.T).float().to(device)
        
        anchor_dot_contrast = torch.div(
            torch.matmul(features, features.T),
            self.temperature
        )
        
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()
        
        logits_mask = torch.ones_like(mask) - torch.eye(batch_size, device=device)
        mask = mask * logits_mask
        
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True) + 1e-12)
        
        mean_log_prob_pos = (mask * log_prob).sum(1) / (mask.sum(1) + 1e-12)
        
        weights = torch.where(labels.squeeze() == 1, self.minority_weight, 1.0).to(device)
        loss = -(weights * mean_log_prob_pos).mean()
        
        return loss

class CombinedLoss(nn.Module):
    def __init__(self, classification_loss, contrastive_loss, alpha=0.3):
        super(CombinedLoss, self).__init__()
        self.classification_loss = classification_loss
        self.contrastive_loss = contrastive_loss
        self.alpha = alpha
        
    def forward(self, logits, embeddings, labels):
        cls_loss = self.classification_loss(logits.squeeze(), labels.float())
        cont_loss = self.contrastive_loss(embeddings, labels)
        total_loss = cls_loss + self.alpha * cont_loss
        return total_loss, cls_loss, cont_loss

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EnhancedCNNBinaryClassifier(
    input_shape=(3, 210, 70), 
    physical_feature_dim=len(train_dataset.feature_columns)
).to(device)
classification_loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(6.0).to(device))
contrastive_loss = SupervisedContrastiveLoss(temperature=0.05, minority_weight=2.0)
criterion = CombinedLoss(
    classification_loss=classification_loss,
    contrastive_loss=contrastive_loss,
    alpha=0.3
)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-6)

num_epochs = 100
threshold = 0.5

In [16]:
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score


best_val_loss = float('inf')
best_uar = 0.0
patience = 6
patience_counter = 0
training_losses = []
validation_losses = []
start_time = time.time()
early_stop_counter  = 0

print("Starting training...\n")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_cls_loss = 0.0
    running_cont_loss = 0.0
    all_preds, all_labels = [], []

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")

    for image, physical_features, label_num in progress_bar:
        batch_X = image.to(device)
        batch_y = label_num.to(device)
        physical_features = physical_features.to(device)

        optimizer.zero_grad()
        
        embeddings, logits = model(batch_X, physical_features)
        
        total_loss, cls_loss, cont_loss = criterion(logits, embeddings, batch_y)

        total_loss.backward()
        optimizer.step()

        preds = (torch.sigmoid(logits.squeeze()) > threshold).long()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

        running_loss += total_loss.item()
        running_cls_loss += cls_loss.item()
        running_cont_loss += cont_loss.item()
        
        progress_bar.set_postfix({
            'total_loss': f'{total_loss.item():.4f}',
            'cls_loss': f'{cls_loss.item():.4f}',
            'cont_loss': f'{cont_loss.item():.4f}'
        })
    epoch_loss = running_loss / len(train_loader)
    epoch_cls_loss = running_cls_loss / len(train_loader)
    epoch_cont_loss = running_cont_loss / len(train_loader)
    
    print(f"📊 Epoch {epoch+1} Losses:")
    print(f"  Total: {epoch_loss:.4f}, Classification: {epoch_cls_loss:.4f}, Contrastive: {epoch_cont_loss:.4f}")
    
    train_accuracy = accuracy_score(all_labels, all_preds)
    train_uar = recall_score(all_labels, all_preds, average='macro')

    model.eval()
    val_loss, val_preds, val_labels = 0.0, [], []

    with torch.no_grad():
        for image, physical_features, label_num in tqdm(devel_loader, desc="Validating"):
            batch_X = image.to(device)
            physical_features = physical_features.to(device)
            batch_y = label_num.to(device)

            embeddings, logits = model(batch_X, physical_features)
            
            total_loss, cls_loss, cont_loss = criterion(logits, embeddings, batch_y)
            val_loss += total_loss.item()

            preds = (torch.sigmoid(logits.squeeze()) > threshold).long()
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(batch_y.cpu().numpy())

    avg_val_loss = val_loss / len(devel_loader)
    validation_losses.append(avg_val_loss)

    val_accuracy = accuracy_score(val_labels, val_preds)
    val_uar = recall_score(val_labels, val_preds, average='macro')
    

    print(f"\nEpoch [{epoch+1}] Summary:")
    print(f"  📈 Training   - Loss: {epoch_loss:.4f}, Acc: {train_accuracy:.4f}, UAR: {train_uar:.4f}")
    print(f"  📊 Validation - Loss: {avg_val_loss:.4f}, Acc: {val_accuracy:.4f}, UAR: {val_uar:.4f}")
    print(f"  📦 Processed  - Train: {len(all_labels)} samples, Val: {len(val_labels)} samples")

    if val_uar > best_uar:
        best_uar = val_uar
        early_stop_counter = 0
        torch.save(model.state_dict(), "best_cv_fusion.pth")
        print(f"🌟 New best UAR: {best_uar:.4f}, saving model...")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"No improvement in UAR for {patience} epochs, early stopping...")
            break

print(f"\n🎉 Training complete in {(time.time() - start_time)/60:.2f} min")
print(f"Best Validation UAR: {best_uar:.4f}")

Starting training...



Epoch 1 Training: 100%|██████████| 274/274 [01:10<00:00,  3.88it/s, total_loss=3.1334, cls_loss=1.6805, cont_loss=4.8432]


📊 Epoch 1 Losses:
  Total: 3.0151, Classification: 1.5003, Contrastive: 5.0496


Validating: 100%|██████████| 150/150 [01:04<00:00,  2.33it/s]



Epoch [1] Summary:
  📈 Training   - Loss: 3.0151, Acc: 0.6548, UAR: 0.5427
  📊 Validation - Loss: 2.4275, Acc: 0.7258, UAR: 0.6395
  📦 Processed  - Train: 8768 samples, Val: 9596 samples
🌟 New best UAR: 0.6395, saving model...


...


Epoch [6] Summary:
  📈 Training   - Loss: 1.2986, Acc: 0.9528, UAR: 0.9671
  📊 Validation - Loss: 2.7829, Acc: 0.8780, UAR: 0.5186
  📦 Processed  - Train: 8768 samples, Val: 9596 samples


Epoch 7 Training: 100%|██████████| 274/274 [01:08<00:00,  3.98it/s, total_loss=1.0618, cls_loss=0.0712, cont_loss=3.3018]


📊 Epoch 7 Losses:
  Total: 1.2112, Classification: 0.1825, Contrastive: 3.4290


Validating: 100%|██████████| 150/150 [01:02<00:00,  2.38it/s]


Epoch [7] Summary:
  📈 Training   - Loss: 1.2112, Acc: 0.9689, UAR: 0.9778
  📊 Validation - Loss: 2.9361, Acc: 0.8820, UAR: 0.5187
  📦 Processed  - Train: 8768 samples, Val: 9596 samples
No improvement in UAR for 6 epochs, early stopping...

🎉 Training complete in 15.46 min
Best Validation UAR: 0.6395
